In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime,timedelta

import yfinance as yf
from statsmodels.tsa.stattools import adfuller

### Part 1 — finding half_life, ADF, hurst , market type ,...

In [1]:
import os
import pandas as pd
import numpy as np
import requests
import yfinance as yf
from datetime import datetime, timedelta
from statsmodels.tsa.stattools import adfuller
from statsmodels.api import add_constant, OLS
import contextlib
import io

# Configuration
DATA_FOLDER = "crypto_daily_data"
os.makedirs(DATA_FOLDER, exist_ok=True)

START_DATE = "2024-01-01"
END_DATE = "2026-01-01"

# Market regime parameters
EMA_LEN = 200
ATR_LEN = 14
MIN_TREND_SLOPE = 0.0005  # Minimum slope for uptrend
MAX_RANGE_SLOPE = 0.0003  # Maximum slope for range market
MIN_ATR_PCT = 0.02  # Minimum ATR% for range market
MIN_ABOVE_EMA = 0.6  # Minimum percentage above EMA200
GRID_ATR_K = 0.5  # Grid spacing multiplier based on ATR

def get_top50_coins(vs_currency="usd", per_page=50):
    """Get top 50 cryptocurrencies by market cap"""
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": vs_currency,
        "order": "market_cap_desc",
        "per_page": per_page,
        "page": 1,
        "sparkline": "false"
    }
    request = requests.get(url, params=params, timeout=30)
    request.raise_for_status()
    data = request.json()
    df = pd.DataFrame(data)

    return (
        df[["symbol", "market_cap", "market_cap_rank"]]
        .sort_values("market_cap_rank")
        .reset_index(drop=True)
    )

def to_yahoo_ticker(symbol: str) -> str:
    """Convert symbol to Yahoo Finance ticker format"""
    s = symbol.upper().strip()
    if s.endswith("-USD"):
        return s
    return f"{s}-USD"

def download_and_save_daily_data(ticker: str, start_date: str, end_date: str) -> pd.DataFrame:
    """Download daily OHLC data from Yahoo Finance and save to file"""
    end_dt = datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=1)
    end_plus_one = end_dt.strftime("%Y-%m-%d")
    
    buf = io.StringIO()
    with contextlib.redirect_stdout(buf), contextlib.redirect_stderr(buf):
        df = yf.download(
            ticker,
            start=start_date,
            end=end_plus_one,
            interval="1d",
            auto_adjust=False,
            progress=False,
            threads=True,
        )
    
    if df is None or len(df) == 0:
        return pd.DataFrame()

    if isinstance(df.columns, pd.MultiIndex):
        df = df.xs(ticker, axis=1, level=1, drop_level=True)

    df = df.rename(columns=str.title)
    
    # Convert to numeric
    for col in ["Open", "High", "Low", "Close"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").squeeze()
    
    # Save data to file
    if not df.empty:
        filename = f"{ticker.replace('-', '_')}_daily.csv"
        filepath = os.path.join(DATA_FOLDER, filename)
        df.to_csv(filepath)
        print(f"✅ Data saved for {ticker} at {filepath}")
    
    return df.dropna(subset=["Open", "High", "Low", "Close"])

def calc_atr(df, n=14):
    """Calculate Average True Range"""
    high = df["High"]
    low = df["Low"]
    close = df["Close"]
    
    tr = pd.concat([
        high - low,
        (high - close.shift()).abs(),
        (low - close.shift()).abs()
    ], axis=1).max(axis=1)
    
    return tr.ewm(alpha=1/n, adjust=False).mean()

def half_life(series: pd.Series) -> float:
    """Calculate half-life of mean reversion"""
    series = series.dropna()
    if len(series) < 10:
        return np.nan
    
    price_lag = series.shift(1)
    price_diff = series - price_lag

    valid = pd.concat([price_lag, price_diff], axis=1).dropna()
    if len(valid) < 5:
        return np.nan
    
    price_lag_clean = valid.iloc[:, 0]
    price_diff_clean = valid.iloc[:, 1]

    X = add_constant(price_lag_clean)
    try:
        model = OLS(price_diff_clean, X).fit()
        beta = model.params.iloc[1]
        if (1 + beta) > 0:
            hl = -np.log(2) / np.log(1 + beta)
            return float(hl)
        else:
            return np.nan
    except:
        return np.nan

def hurst_exponent(series, q=2.0, max_lag=None, min_lag=2) -> float:
    """Calculate Hurst exponent"""
    x = np.asarray(series, dtype=float)
    x = x[~np.isnan(x)]
    n = len(x)
    
    if n < 50:
        return np.nan
    
    if max_lag is None:
        max_lag = n // 4
    
    if max_lag <= min_lag:
        return np.nan
    
    lags = np.arange(min_lag, max_lag)
    K = np.zeros_like(lags, dtype=float)
    
    for i, lag in enumerate(lags):
        diffs = np.abs(x[lag:] - x[:-lag])
        K[i] = np.mean(diffs ** q)
    
    log_lags = np.log(lags)
    log_K = np.log(K)
    
    try:
        slope, _ = np.polyfit(log_lags, log_K, 1)
        H = slope / q
        return float(H)
    except:
        return np.nan

def adf_pvalue(series: pd.Series):
    """Run Augmented Dickey-Fuller test"""
    s = series.dropna()
    if len(s) < 10:
        return np.nan, np.nan
    
    try:
        test_stat, p_value = adfuller(s)[:2]
        return test_stat, p_value
    except:
        return np.nan, np.nan

def analyze_market_regime(df):
    """Analyze market regime similar to the provided code"""
    if len(df) < 300:
        return None
    
    df = df.copy()
    
    # Calculate technical indicators
    df["ema200"] = df["Close"].ewm(span=EMA_LEN, adjust=False).mean()
    df["atr"] = calc_atr(df, ATR_LEN)
    df["atr_pct"] = df["atr"] / df["Close"]
    
    df = df.dropna()
    if df.empty:
        return None
    
    # Focus on 2025 period as in the original code
    df.index = pd.to_datetime(df.index)
    analysis_start = '2025-01-01'
    analysis_end = '2025-12-01'
    
    mask = (df.index >= analysis_start) & (df.index <= analysis_end)
    period_df = df.loc[mask].copy()
    
    if len(period_df) < 60:
        return None
    
    # Get 2025 data
    close_2025 = period_df["Close"]
    ema_2025 = period_df["ema200"]
    
    # Calculate percentage above EMA200
    pct_above = (close_2025 > ema_2025).mean()
    
    # Calculate trend slope
    y = np.log(close_2025.values)
    x = np.arange(len(y))
    if len(y) > 1:
        slope = np.cov(x, y, bias=True)[0, 1] / np.var(x)
    else:
        slope = 0
    
    # Calculate recent ATR%
    atr_pct_recent = period_df["atr_pct"].tail(60).mean()
    
    # Get latest values
    last_close = df["Close"].iloc[-1]
    last_ema = df["ema200"].iloc[-1]
    last_atr = df["atr"].iloc[-1]
    
    # Determine market regime
    if slope > MIN_TREND_SLOPE and last_close > last_ema:
        regime = "UPTREND"
    elif abs(slope) < MAX_RANGE_SLOPE and atr_pct_recent > MIN_ATR_PCT:
        regime = "RANGE"
    else:
        regime = "OTHER"
    
    # Calculate grid spacing
    grid_spacing = GRID_ATR_K * last_atr
    
    # Selection criteria
    selected = (
        regime in ["UPTREND", "RANGE"]
        and pct_above >= MIN_ABOVE_EMA
        and last_close > last_ema
    )
    
    return {
        "last_close": float(last_close),
        "ema200": float(last_ema),
        "pct_above_ema200": float(pct_above),
        "atr": float(last_atr),
        "atr_pct": float(atr_pct_recent),
        "trend_slope": float(slope),
        "regime": regime,
        "grid_spacing": float(grid_spacing),
        "grid_spacing_pct": float(grid_spacing / last_close),
        "selected": selected
    }

# Main processing
print("📊 Starting analysis of top 50 cryptocurrencies...")
print(f"📁 Data will be saved in folder: '{DATA_FOLDER}'")

top50 = get_top50_coins()
print(f"✅ Retrieved {len(top50)} cryptocurrencies")

results = []

for idx, row in top50.iterrows():
    symbol = row["symbol"]
    rank = row["market_cap_rank"]
    ticker = to_yahoo_ticker(symbol)
    
    print(f"\n[{idx+1}/{len(top50)}] Analyzing {ticker} (Rank: {rank})...")
    
    try:
        # Download and save data
        df = download_and_save_daily_data(ticker, START_DATE, END_DATE)
        
        if df.empty:
            print(f"  ⚠️ No data found for {ticker}")
            continue
        
        # Calculate ADF test
        adf_stat, adf_p = adf_pvalue(df["Close"])
        
        # Calculate half-life
        hl_value = half_life(df["Close"])
        
        # Calculate Hurst exponent
        hurst_value = hurst_exponent(df["Close"])
        
        # Analyze market regime
        market_stats = analyze_market_regime(df)
        
        if market_stats is None:
            print(f"  ⚠️ Insufficient data for market regime analysis on {ticker}")
            continue
        
        # Calculate additional range metrics
        price_range = df["Close"].max() - df["Close"].min()
        range_pct = (price_range / df["Close"].min()) * 100 if df["Close"].min() != 0 else np.nan
        
        # Calculate average daily range
        daily_ranges = (df["High"] - df["Low"]) / df["Close"].shift(1)
        avg_daily_range = daily_ranges.mean() * 100 if len(daily_ranges) > 0 else np.nan
        
        # Determine market state based on Hurst
        if pd.notna(hurst_value):
            if hurst_value > 0.65:
                hurst_state = "Trending"
            elif hurst_value < 0.35:
                hurst_state = "Mean-Reverting"
            else:
                hurst_state = "Random"
        else:
            hurst_state = "Unknown"
        
        # Combine all results
        result = {
            "rank": rank,
            "symbol": symbol.upper(),
            "ticker": ticker,
            "days_data": len(df),
            "last_price": float(df["Close"].iloc[-1]),
            
            # ADF test results
            "adf_statistic": adf_stat,
            "adf_p_value": adf_p,
            "stationary_95pct": adf_p < 0.05 if pd.notna(adf_p) else False,
            
            # Range metrics
            "price_range": float(price_range),
            "range_pct": float(range_pct),
            "avg_daily_range_pct": float(avg_daily_range),
            
            # Mean reversion metrics
            "half_life": hl_value,
            "hurst_exponent": hurst_value,
            "hurst_state": hurst_state,
            
            # Market regime analysis
            "regime": market_stats["regime"],
            "trend_slope": market_stats["trend_slope"],
            "pct_above_ema200": market_stats["pct_above_ema200"],
            "atr": market_stats["atr"],
            "atr_pct": market_stats["atr_pct"],
            "grid_spacing": market_stats["grid_spacing"],
            "grid_spacing_pct": market_stats["grid_spacing_pct"],
            "selected": market_stats["selected"]
        }
        
        results.append(result)
        print(f"  ✅ Analysis complete - {len(df)} days of data")
        
    except Exception as e:
        print(f"  ❌ Error analyzing {ticker}: {str(e)}")
        continue

# Create results DataFrame
if results:
    results_df = pd.DataFrame(results)
    
    # Sort by selected status and regime
    results_df = results_df.sort_values(
        ["selected", "regime", "pct_above_ema200", "atr_pct"],
        ascending=[False, True, False, False]
    ).reset_index(drop=True)
    
    # Save full results
    results_file = os.path.join(DATA_FOLDER, "crypto_comprehensive_analysis.csv")
    results_df.to_csv(results_file, index=False)
    
    # Create display version
    results_df_display = results_df.copy()
    
    # Format numeric columns for display
    float_cols = ["adf_statistic", "adf_p_value", "last_price", "price_range", 
                  "range_pct", "avg_daily_range_pct", "half_life", "hurst_exponent",
                  "trend_slope", "pct_above_ema200", "atr", "atr_pct", 
                  "grid_spacing", "grid_spacing_pct"]
    
    for col in float_cols:
        if col in results_df_display.columns:
            results_df_display[col] = results_df_display[col].apply(
                lambda x: f"{x:,.6f}" if pd.notna(x) else "N/A"
            )
    
    # Display results
    print(f"\n{'='*60}")
    print("✅ ANALYSIS COMPLETE!")
    print(f"📁 Raw data saved in folder: {DATA_FOLDER}/")
    print(f"📊 Results saved to: {results_file}")
    print(f"{'='*60}")
    
    # Show selected symbols for grid trading
    selected_symbols = results_df.loc[results_df["selected"], "symbol"].tolist()
    print(f"\n📈 Selected symbols for Grid Trading ({len(selected_symbols)}):")
    print(selected_symbols)
    
    # Display full table
    print("\n📋 Full Analysis Results:")
    display_cols = ["symbol", "regime", "selected", "pct_above_ema200", 
                    "atr_pct", "hurst_exponent", "half_life", "stationary_95pct"]
    print(results_df_display[display_cols].to_string(index=False))
    
    # Generate summary statistics
    print("\n================ SUMMARY ================")
    
    total_symbols = len(top50)
    analyzed = len(results_df)
    selected_count = int(results_df["selected"].sum()) if not results_df.empty else 0
    
    print(f"Universe size (Top 50): {total_symbols}")
    print(f"Downloaded & analyzed : {analyzed}")
    print(f"Selected for Grid     : {selected_count}")
    
    if not results_df.empty:
        print("\n--- Market Regime Breakdown ---")
        print(results_df["regime"].value_counts())
        
        print("\n--- Hurst State Breakdown ---")
        print(results_df["hurst_state"].value_counts())
        
        print("\n--- Stationarity Breakdown ---")
        print(f"Stationary (95% confidence): {results_df['stationary_95pct'].sum()}")
        print(f"Non-stationary: {len(results_df) - results_df['stationary_95pct'].sum()}")
        
        print("\n--- Selected Symbols (UPTREND / RANGE) ---")
        selected_df = results_df.loc[results_df["selected"], 
                                     ["symbol", "regime", "pct_above_ema200", 
                                      "atr_pct", "grid_spacing_pct", "hurst_state"]]
        print(selected_df.sort_values("atr_pct", ascending=False).to_string(index=False))
        
        print("\n--- Top 5 Most Volatile (ATR %) ---")
        top_volatile = results_df.sort_values("atr_pct", ascending=False).head(5)
        print(top_volatile[["symbol", "regime", "atr_pct", "grid_spacing_pct"]].to_string(index=False))
        
        print("\n--- Top 5 Mean-Reverting (Lowest Half-Life) ---")
        top_mr = results_df.sort_values("half_life", ascending=True).head(5)
        print(top_mr[["symbol", "half_life", "hurst_state", "atr_pct"]].to_string(index=False))
    
    print("=========================================\n")
    
    # Save summary report
    summary_file = os.path.join(DATA_FOLDER, "analysis_summary.txt")
    with open(summary_file, 'w') as f:
        f.write("CRYPTO ANALYSIS SUMMARY\n")
        f.write("=" * 50 + "\n")
        f.write(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Period: {START_DATE} to {END_DATE}\n")
        f.write(f"Total symbols analyzed: {analyzed}/{total_symbols}\n")
        f.write(f"Selected for grid: {selected_count}\n")
        f.write("\nSelected Symbols:\n")
        for symbol in selected_symbols:
            f.write(f"- {symbol}\n")
    
    print(f"📝 Summary report saved to: {summary_file}")
    
else:
    print("⚠️ No results generated!")

print("🎉 Processing complete!")

📊 Starting analysis of top 50 cryptocurrencies...
📁 Data will be saved in folder: 'crypto_daily_data'
✅ Retrieved 50 cryptocurrencies

[1/50] Analyzing BTC-USD (Rank: 1)...
✅ Data saved for BTC-USD at crypto_daily_data\BTC_USD_daily.csv
  ✅ Analysis complete - 732 days of data

[2/50] Analyzing ETH-USD (Rank: 2)...
✅ Data saved for ETH-USD at crypto_daily_data\ETH_USD_daily.csv
  ✅ Analysis complete - 732 days of data

[3/50] Analyzing USDT-USD (Rank: 3)...
✅ Data saved for USDT-USD at crypto_daily_data\USDT_USD_daily.csv
  ✅ Analysis complete - 732 days of data

[4/50] Analyzing BNB-USD (Rank: 4)...
✅ Data saved for BNB-USD at crypto_daily_data\BNB_USD_daily.csv
  ✅ Analysis complete - 732 days of data

[5/50] Analyzing XRP-USD (Rank: 5)...
✅ Data saved for XRP-USD at crypto_daily_data\XRP_USD_daily.csv
  ✅ Analysis complete - 732 days of data

[6/50] Analyzing USDC-USD (Rank: 6)...
✅ Data saved for USDC-USD at crypto_daily_data\USDC_USD_daily.csv
  ✅ Analysis complete - 732 days of d

C:\Users\T.Abdellahi\AppData\Local\Temp\ipykernel_13284\1621600575.py:130: RuntimeWarning: divide by zero encountered in scalar divide
  hl = -np.log(2) / np.log(1 + beta)
C:\Users\T.Abdellahi\AppData\Local\Temp\ipykernel_13284\1621600575.py:160: RuntimeWarning: divide by zero encountered in log
  log_K = np.log(K)


✅ Data saved for XAUT-USD at crypto_daily_data\XAUT_USD_daily.csv
  ✅ Analysis complete - 732 days of data

[33/50] Analyzing PYUSD-USD (Rank: 33)...
✅ Data saved for PYUSD-USD at crypto_daily_data\PYUSD_USD_daily.csv
  ✅ Analysis complete - 732 days of data

[34/50] Analyzing TON-USD (Rank: 34)...
✅ Data saved for TON-USD at crypto_daily_data\TON_USD_daily.csv
  ✅ Analysis complete - 732 days of data

[35/50] Analyzing CRO-USD (Rank: 35)...
✅ Data saved for CRO-USD at crypto_daily_data\CRO_USD_daily.csv
  ✅ Analysis complete - 732 days of data

[36/50] Analyzing RAIN-USD (Rank: 36)...
✅ Data saved for RAIN-USD at crypto_daily_data\RAIN_USD_daily.csv
  ✅ Analysis complete - 732 days of data

[37/50] Analyzing M-USD (Rank: 37)...
✅ Data saved for M-USD at crypto_daily_data\M_USD_daily.csv
  ✅ Analysis complete - 732 days of data

[38/50] Analyzing DOT-USD (Rank: 38)...
✅ Data saved for DOT-USD at crypto_daily_data\DOT_USD_daily.csv
  ✅ Analysis complete - 732 days of data

[39/50] Analy

In [3]:
results_df_display

,rank,symbol,ticker,days_data,last_price,adf_statistic,adf_p_value,stationary_95pct,price_range,range_pct,...,hurst_exponent,hurst_state,regime,trend_slope,pct_above_ema200,atr,atr_pct,grid_spacing,grid_spacing_pct,selected
0,11,WBT,WBT-USD,732,56.806820,0.121994,0.967531,False,57.142474,985.987391,...,0.549143,Random,UPTREND,0.002320,1.000000,1.381762,0.032744,0.690881,0.012162,True
1,41,PAXG,PAXG-USD,732,"4,356.462891",0.610275,0.987866,False,"2,585.504639",130.428663,...,0.625840,Random,UPTREND,0.001231,1.000000,59.041175,0.018444,29.520587,0.006776,True
2,32,XAUT,XAUT-USD,732,"4,349.374512",0.657868,0.988958,False,"2,551.508545",128.092221,...,0.633490,Random,UPTREND,0.001244,1.000000,55.033352,0.016286,27.516676,0.006327,True
3,17,XMR,XMR-USD,732,419.580536,-0.294600,0.926266,False,371.688957,354.065300,...,0.403808,Random,UPTREND,0.001675,0.898507,22.227701,0.063630,11.113851,0.026488,True
4,13,BCH,BCH-USD,732,592.327026,-2.360625,0.153159,False,465.616882,202.844709,...,0.356983,Random,UPTREND,0.001692,0.650746,26.518934,0.057199,13.259467,0.022385,True
5,50,HTX,HTX-USD,709,0.000002,-1.572565,0.497359,False,0.000003,300.000000,...,0.308570,Mean-Reverting,OTHER,-0.000106,1.000000,0.000000,0.003337,0.000000,0.000009,False
6,43,USDF,USDF-USD,732,2.142394,-2.058526,0.261515,False,1.577566,173.805061,...,0.410947,Random,OTHER,0.000009,1.000000,0.000000,0.000000,0.000000,0.000000,False
7,15,LEO,LEO-USD,732,9.567177,-1.627142,0.468987,False,6.074466,157.035015,...,0.542400,Random,OTHER,-0.000025,0.907463,0.432279,0.016785,0.216140,0.022592,False
8,8,TRX,TRX-USD,732,0.286523,-1.184759,0.680117,False,0.323224,314.518164,...,0.456997,Random,OTHER,0.001162,0.880597,0.004203,0.024591,0.002102,0.007335,False
9,42,BGB,BGB-USD,732,3.499719,-1.834719,0.363366,False,7.526230,"1,302.746960",...,0.517747,Random,OTHER,-0.000860,0.817910,0.061324,0.048733,0.030662,0.008761,False


## Part 2: 